<a href="https://colab.research.google.com/github/wadiazu/GEEE/blob/main/Data_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Este codigo fue diseñado para recopilar información referente a Precipitación y temperatura -->

###### Elaborado por: 

###### Wilson Arbey Diaz Urueña , wadiazu@unal.edu.co
###### Luis Jesus Martinez Hernandez, lujmartinezhe@unal.edu.co

In [ ]:
# Importar la API (Application Programming Interfaces) de Engine. Para conectar Colab con GEE
import ee

In [ ]:
# Importar librerias en caso que sean necesarias
import pandas as pd
import numpy as np
import datetime
import glob
import matplotlib.pyplot as plt
import functools
from osgeo import gdal
from datetime import datetime
from datetime import timedelta

!pip install geopandas
!pip install descartes 

import geopandas as gpd

     |████████████████████████████████| 1.0MB 3.9MB/s 
     |████████████████████████████████| 6.5MB 21.4MB/s 
     |████████████████████████████████| 15.3MB 222kB/s 


In [ ]:
# Permisos de acceso. Cundo se tiene la información en el drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Para creear el acceso con Engine y autenticar usuario
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Nv7LOzoaVu35baVzU6o7y9s-_6F9_UL90dUdoViX6LY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5FcJ6Q8VOGV0xcXgH2vqcXEzE3-NWUBVyz2JMnJv8L7TWHYaMU8Do

Successfully saved authorization token.


# Precipitation

In [ ]:
Chirps=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

In [ ]:
med= ee.FeatureCollection('users/wadiazu/Medellin')
medur= ee.FeatureCollection('users/wadiazu/Medellin_urbano')
medru= ee.FeatureCollection('users/wadiazu/Medellin_rural')

In [ ]:
Med=med.geometry()
Medur=medur.geometry()
Medru=medru.geometry()

In [ ]:
months = ee.List.sequence(1,12) 
years = ee.List.sequence(2007,2017)

In [ ]:
#definimos las fechas de inicio y fin
inicio=datetime(2006,12,31,0,0,0)
fin=datetime(2007,1,28,0,0,0)

In [ ]:
#lista de fechas segun la frecuencia, diario, mensual
Daterange=pd.date_range(inicio,fin,freq="d")
print(Daterange)

DatetimeIndex(['2006-12-31', '2007-01-01', '2007-01-02', '2007-01-03',
               '2007-01-04', '2007-01-05', '2007-01-06', '2007-01-07',
               '2007-01-08', '2007-01-09', '2007-01-10', '2007-01-11',
               '2007-01-12', '2007-01-13', '2007-01-14', '2007-01-15',
               '2007-01-16', '2007-01-17', '2007-01-18', '2007-01-19',
               '2007-01-20', '2007-01-21', '2007-01-22', '2007-01-23',
               '2007-01-24', '2007-01-25', '2007-01-26', '2007-01-27',
               '2007-01-28'],
              dtype='datetime64[ns]', freq='D')


In [ ]:
lista=[]
for i in Daterange:
  image=Chirps.select(['precipitation']).filterDate(i,i+ timedelta(days=1)).sum().clipToCollection(med)
  lista.append(image)

In [ ]:
chirps_mean=ee.ImageCollection(lista)

In [ ]:
months = ee.List.sequence(1,12) 
years = ee.List.sequence(2007,2017)

In [ ]:
chirps_mean = ee.ImageCollection.fromImages(years.map(lambda y: months.map(lambda m: CHIRPS.filter(ee.Filter.calendarRange(m,m,'month')).
                                                                           filter(ee.Filter.calendarRange(y,y,'year')).sum().set('month', m).set('year', y).
                                                                           set('date', ee.Date.fromYMD(y,m,1)).set('system:time_start',ee.Date.fromYMD(y,m,1)))).flatten())


In [ ]:
def means(img, arg_1):
    precip = img.reduceRegion(ee.Reducer.mean(),arg_1,5500)
    props = {'precipitation': precip.get('precipitation')}
    return ee.Feature(arg_1, props)

In [ ]:
def fc_to_dict(med):
  prop_names = med.first().propertyNames()
  prop_lists = med.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')
  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
func_co = functools.partial(means, arg_1=Med)
chirps = pd.DataFrame(fc_to_dict(ee.FeatureCollection(chirps_mean.map(func_co))).getInfo()['precipitation'], columns=['Medellin'])
func_ve = functools.partial(means, arg_1=Medur)
chirps['MedellinRural'] = pd.DataFrame(fc_to_dict(ee.FeatureCollection(chirps_mean.map(func_ve))).getInfo()['precipitation'])
func_wa = functools.partial(means, arg_1=Medru)
chirps['MedellinUrbano'] = pd.DataFrame(fc_to_dict(ee.FeatureCollection(chirps_mean.map(func_wa))).getInfo()['precipitation'])


In [ ]:
print(chirps)

     Medellin  MedellinRural  MedellinUrbano
0    4.689406       6.809430        3.655764
1    0.000000       0.000000        0.000000
2    0.000000       0.000000        0.000000
3    0.000000       0.000000        0.000000
4    5.711017       5.123229        5.968616
5    0.000000       0.000000        0.000000
6    0.000000       0.000000        0.000000
7    0.000000       0.000000        0.000000
8   13.114437      14.097693       12.645848
9    0.000000       0.000000        0.000000
10   0.000000       0.000000        0.000000
11   0.000000       0.000000        0.000000
12   0.000000       0.000000        0.000000
13   6.396178       6.084783        6.507294
14   0.000000       0.000000        0.000000
15   0.000000       0.000000        0.000000
16   0.959090       0.412804        1.189793
17   1.077423       0.412804        1.361648
18   0.000000       0.000000        0.000000
19   0.000000       0.000000        0.000000
20   0.000000       0.000000        0.000000
21   0.000

In [ ]:
chirps.index=Daterange

In [ ]:
print(chirps)

            Medellin  MedellinRural  MedellinUrbano
2006-12-31  4.689406       6.809430        3.655764
2007-01-01  0.000000       0.000000        0.000000
2007-01-02  0.000000       0.000000        0.000000
2007-01-03  0.000000       0.000000        0.000000
2007-01-04  5.711017       5.123229        5.968616
...              ...            ...             ...
2019-12-24  0.002122       0.002164        0.000000
2019-12-25  5.450239       4.487188        5.873099
2019-12-26  4.757602       5.540309        4.381318
2019-12-27  7.033531       5.792221        7.565150
2019-12-28  0.000000       0.000000        0.000000

[4746 rows x 3 columns]


In [ ]:
Datos_semanal=chirps.resample("7D").sum()

In [ ]:
print(Datos_semanal)

             Medellin  MedellinRural  MedellinUrbano
2006-12-31  10.400423      11.932660        9.624380
2007-01-07  19.510615      20.182476       19.153142
2007-01-14   2.036514       0.825607        2.551441
2007-01-21  19.405491      21.616968       18.439284
2007-01-28   0.000000       0.000000        0.000000
...               ...            ...             ...
2019-11-24  15.607313      15.862694       15.462729
2019-12-01   9.035788       6.591552       10.151920
2019-12-08  26.450263      26.978917       26.226603
2019-12-15  13.327096      14.847955       12.606804
2019-12-22  17.386120      15.821882       18.026700

[678 rows x 3 columns]


In [ ]:
#exportar a csv
# Datos_semanal.to_csv('Datos_semanal.csv')

#Temperatura

In [ ]:
LST=ee.ImageCollection("MODIS/006/MOD11A1")

In [ ]:
# lst=LST.select(['LST_Day_1km'])
# # lst_scale=lst.multiply(0.02)

In [ ]:
med= ee.FeatureCollection('users/wadiazu/Medellin')
medur= ee.FeatureCollection('users/wadiazu/Medellin_urbano')
medru= ee.FeatureCollection('users/wadiazu/Medellin_rural')

In [ ]:
Med=med.geometry()
Medur=medur.geometry()
Medru=medru.geometry()

In [ ]:
#definimos las fechas de inicio y fin
inicio=datetime(2006,12,31,0,0,0)
fin=datetime(2007,1,1,0,0,0)

In [ ]:
#lista de fechas segun la frecuencia, diario, mensual
Daterange=pd.date_range(inicio,fin,freq="d")
print(Daterange)

DatetimeIndex(['2006-12-31', '2007-01-01'], dtype='datetime64[ns]', freq='D')


In [ ]:
lista=[]
for i in Daterange:
   image=LST.select(['LST_Day_1km']).filterDate(i,i+ timedelta(days=1)).sum().clipToCollection(med)
   lista.append(image)

In [ ]:
lst_mean=ee.ImageCollection.fromImages(lista)

In [ ]:
months = ee.List.sequence(1,12) 
years = ee.List.sequence(2007,2017)

In [ ]:
lst_mean = ee.ImageCollection.fromImages(years.map(lambda y: months.map(lambda m: LST.filter(ee.Filter.calendarRange(m,m,'month')).
                                                                           filter(ee.Filter.calendarRange(y,y,'year')).sum().set('month', m).set('year', y).
                                                                           set('date', ee.Date.fromYMD(y,m,1)).set('system:time_start',ee.Date.fromYMD(y,m,1)))).flatten())


In [ ]:
def means(img, arg_1):
    temp = img.reduceRegion(ee.Reducer.mean(),arg_1,1000)
    props = {'LST_Day_1km': temp.get('LST_Day_1km')}
    return ee.Feature(arg_1, props)

In [ ]:
def fc_to_dict(med):
  prop_names = med.first().propertyNames()
  prop_lists = med.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')
  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
func_co = functools.partial(means, arg_1=Med)
chirps = pd.DataFrame(fc_to_dict(ee.FeatureCollection(lst_mean.map(func_co))).getInfo()['LST_Day_1km'], columns=['Medellin'])

In [ ]:
print(chirps)


          Medellin
0     61440.862151
1    267944.408003
2     49216.941840
3     37182.875673
4     78413.262620
..             ...
127  123828.776469
128   94201.975344
129   72497.541278
130   78496.264270
131  141683.041559

[132 rows x 1 columns]


##GEE ENSAYO

In [ ]:
day=ee.Date.fromYMD(2006,12,31)
enddate=ee.Date.fromYMD(2007,1,28)
difdate =enddate.difference(day, 'day')


In [ ]:
list=ee.List.sequence(0,difdate)


In [ ]:
lista=ee.List.sequence(0,difdate.subtract(1))


In [ ]:
def fechas_fun(number):
  return day.advance(number, 'day')

In [ ]:
fechas_list=list.map(fechas_fun)
print(fechas_list)

In [ ]:
Temp=ee.ImageCollection("MODIS/006/MOD11A1")

In [ ]:
def colection_SL(number):
  return Temp.select(['LST_Day_1km']).filterDate(ee.Date(fechas_list.get(number)), ee.Date(fechas_list.get(number)).advance(1, 'day')).sum().clipToCollection(Med)


In [ ]:
list_Chirps_SL=lista.map(colection_SL)


imagecol_SL=ee.ImageCollection(list_Chirps_SL)


In [ ]:
mean_Dia_SL=imagecol_SL.reduce(ee.Reducer.mean())


In [ ]:
def list_funtion(number):
  image_reduc=ee.Image(list_Chirps_SL.get(number)).reduceRegion(ee.Reducer.mean(),Med,1000)
  listas_ind=[ee.Date(fechas_list.get(number)).format('YYYY-MM-dd'),image_reduc.get("LST_Day_1km")]
  return listas_ind

In [ ]:
Values=lista.map(list_funtion)
print(Values)